# Project Cold with Graphics

In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# Load data 
dft = pd.read_csv('../data/ubc_temperature91_02cl.csv')
dft['Date_Time'] = pd.to_datetime(dft['Date'] + ' ' + dft['Time'])
dft.drop(['Date', 'Time'], axis=1, inplace=True)
dft.set_index('Date_Time', inplace=True)

In [2]:
# Function to calculate days with temperature below cutoff
def calculate_days_below_cutoff(dft1, cutoff):
    # Create a new column for the date part only
    dft1['Date'] = dft1.index.date
    # Group by date and check if any temperature was below the cutoff
    days_below_cutoff = dft1.groupby('Date')['Tair'].apply(lambda x: (x <= cutoff).any())
    # Count the number of days where the condition is True
    num_days_below = days_below_cutoff.sum()
    return int(num_days_below)

# Function to calculate accumulated degree-days below cutoff
def calculate_accumulated_degree_days(dft1, cutoff):
    # Number of hours below the cutoff temperature
    below_hours = (dft1['Tair'] <= cutoff).sum()
    # Convert hours to days
    accumulated_degree_days = below_hours / 24
    return round(accumulated_degree_days, 2)

# Interactive widgets with default values
min_date = dft.index.min().date()
max_date = dft.index.max().date()

start_date = widgets.DatePicker(description='Start Date', value=min_date)
end_date = widgets.DatePicker(description='End Date', value=max_date)
temp_cutoff = widgets.FloatText(description='Temp Cutoff (°C)', value=0.0)

# Metric selection widget
metric_options = ['Days with Temperature Below Cutoff', 'Accumulated Degree-Days Below Cutoff']
metric_selection = widgets.Dropdown(
    options=metric_options,
    value=metric_options[0],
    description='Metric:',
)

output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        # Check if dates are selected
        if not start_date.value or not end_date.value:
            print("Please select both start and end dates.")
            return
        # Ensure dates are strings in the correct format
        start = start_date.value.strftime('%Y-%m-%d')
        end = end_date.value.strftime('%Y-%m-%d')
        # Check if start date is before end date
        if start_date.value > end_date.value:
            print("Start date must be before end date.")
            return
        cutoff = temp_cutoff.value
        # Filter data for the selected period
        dft1 = dft[start:end].copy()
        if dft1.empty:
            print("No data available for the selected period.")
            return
        # Perform calculation based on selected metric
        selected_metric = metric_selection.value
        if selected_metric == 'Days with Temperature Below Cutoff':
            result = calculate_days_below_cutoff(dft1, cutoff)
            result_text = f"Days with Temperature Below Cutoff: {result} days"
        elif selected_metric == 'Accumulated Degree-Days Below Cutoff':
            result = calculate_accumulated_degree_days(dft1, cutoff)
            result_text = f"Accumulated Degree-Days Below Cutoff: {result} days"
        else:
            print("Invalid metric selected.")
            return
        # Display the result
        print(f"**Temperature Analysis**")
        print(f"Period: {start} to {end}")
        print(f"Temperature Cutoff: {cutoff}°C")
        print(result_text)
        
        # Plotting the temperature over the period
        plt.figure(figsize=(12, 6))
        plt.plot(dft1.index, dft1['Tair'], label='Temperature', color='blue')
        plt.axhline(y=cutoff, color='red', linestyle='--', label=f'Cutoff ({cutoff}°C)')
        plt.xlabel('Date')
        plt.ylabel('Temperature (°C)')
        plt.title('Temperature Over Selected Period')
        plt.legend()
        plt.gcf().autofmt_xdate()
        plt.show()
        
# Button to trigger the calculation
calculate_button = widgets.Button(description='Calculate', button_style='success')
calculate_button.on_click(on_button_click)

# Display widgets with a description
intro_text = widgets.HTML(
    value="""
    <h3>Temperature Analysis Tool</h3>
    <p>Select a date range, temperature cutoff, and metric to analyze the temperature data. 
    The plot will show the temperature variations over the selected period.</p>
    """
)

widget_box = widgets.VBox([
    intro_text, 
    start_date, 
    end_date, 
    temp_cutoff, 
    metric_selection, 
    calculate_button, 
    output
])
display(widget_box)
